# Backpropagation

In this lab session, we will code a backpropagation algorithm. This backpropagation algorithm is needed to implement a gradient descent method for a neural network.  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Function to be fitted

We want to approach the following function with a neural network
$$f(x_1,x_2) = x_2 + (1 - 2 x_1 - x_1^2) \sin(\pi x_2)$$

In [ ]:
def f_deterministic(x):
    f = x[2] + (1 - 2*x[1] - x[1]**2) * np.sin(np.pi * x[2])
    return f

In [ ]:
plt.figure()
plt.xlabel('$x_1$', fontsize=16)
plt.ylabel('$x_2$', fontsize=16)
x1 = np.arange(0, 1, 0.01)
x2 = np.arange(0, 1, 0.01)
xx1, xx2 = np.meshgrid(x1, x2)
xx0 = np.ones(xx1.shape)
xx = np.stack((xx0, xx1, xx2), axis=0)
z = f_deterministic(xx)
h = plt.contourf(x1,x2,z,100)
cbar = plt.colorbar(h)

### Neural network
We build a neural network with 4 layers. Each layer has $N_l$ neurons. We use the notations $p^{(l)}$ and $o^{(l)}$ for the predecessor and output vectors of neurons in layer $l$ ($l=0\ldots L$). The matrices of synaptic weights are noted $\Theta^{(l)}$. 

### Feedforward calculation
The feedforward calculation aims at calculating $h_\theta(x)$ using the following feedforward procedure
\begin{align}
o^{(0)} & = x \\
p^{(1)} & = \Theta^{(0)}\cdot o^{(0)}\\
o^{(1)} & = g\left( p^{(1)} \right) \\
p^{(2)} & = \Theta^{(1)}\cdot o^{(1)}\\
o^{(2)} & = g\left( p^{(2)} \right) \\
p^{(3)} & = \Theta^{(2)}\cdot o^{(2)}\\
o^{(3)} & = p^{(3)} \\
h_\theta(x) & = o^{(3)}
\end{align}
where $x=[1, x_1, x_2]$ and $g$ is a non-linear "sigmoid" function. 

In [ ]:
# Number of neurons in each layer
N0 = 3  # INPUT (x0 = 1, x1, x2)
N1 = 8
N2 = 8
N3 = 1  # OUTPUT
Ns = (N0, N1, N2, N3)

In [ ]:
def NLfunction(p):
    f = np.maximum(0,p)
    fprime = 0.5 + 0.5*np.sign(p)
    # f = np.tanh(p)
    # fprime = 1 - np.tanh(p)**2
    # f = 1/(1+np.exp(-p))
    # fprime = f * (1-f)
    return f, fprime

In [ ]:
def NN(Thetas, x):
    o0 = x
    p1 = Thetas[0] @ o0
    o1, gp1 = NLfunction(p1)
    p2 = Thetas[1] @ o1
    o2, gp2  = NLfunction(p2)
    p3 = Thetas[2] @ o2
    o3 = p3
    gp3 = 1
    
    os = (o0, o1, o2, o3)
    gps= (1, gp1, gp2, gp3)
    h = o3
    return h, os, gps    

In [ ]:
# Random initialisation of matrices
Theta0 = np.random.normal(scale=1.0,size=(Ns[1],Ns[0]))
Theta1 = np.random.normal(scale=1.0,size=(Ns[2],Ns[1]))
Theta2 = np.random.normal(scale=1.0,size=(Ns[3],Ns[2]))
Thetas = (Theta0, Theta1, Theta2)

### Question 1
Plot the output of the NN for $x_1 \in [0,1]$ and $x_2 \in [0,1]$.

### Backpropagation algorithm

As we have seen in class, the backpropagation algorithm consists first in calculating the vectors $\delta^{(l)}$ for each layer starting by the output layer $l=L=2$ and calculating $\delta$'s for each layer recursively. 

\begin{align}
\delta^{(L)} & = (h_\theta(x) - y) g'(p^{(L)}) \\
\delta^{(l)} & = \left(\Theta^{(l)}\right)^T \delta^{(l+1)} .* g'\left(p^{(l)}\right)
\end{align}

Once these vectors $\delta$ are calculated the gradients are obtained with this formula
$$D^{(l)}_{ij}\equiv \frac{\partial E}{\partial \Theta_{ij}^{(l)}} = \delta_i^{(l+1)} o_j^{(l)}$$
where $E=\tfrac{1}{2} \left( h_\theta(x) - y \right)^2$ is the quadratic error. 

In [ ]:
# INPUT
x = np.random.rand(3)
x[0]=1
print(x)

### Question 2
Compute the matrices D0, D1 and D2.

In [ ]:
# Compute the matrices here


### Verification of backpropagation algorithm
We want to assess whether the calculation of the gradient ${\partial E}\,/\,{\partial \Theta_{ij}^{(l)}}$ is correct. To do so, we will compare a component of the matrix $D^{(l)}$ with an approximate value of the gradient obtained by calculating 
$$\tilde D_{ij} = \frac{E(\Theta_{ij} + \epsilon) - E(\Theta_{ij})}{\epsilon}$$

### Question 3
Calculate the matrices Dtilde0, Dtilde1 and Dtidle2 and compare them with D0, D1, D2.

In [ ]:
# Calculate the matrices here


### Stochastic gradient descent
We recall the principle of stochastic gradient descent. It is a loop where the following actions are performed. 
- Pick a data point $(x^{(i)},y^{(i)})$
- Calculate the gradient matrices $D^{(l)}$ for this point
- Update the matrices $\Theta^{(l)}$
$$\Theta^{(l)} := \Theta^{(l)} - \alpha \,D^{(l)}$$
where $\alpha$ is the learning parameter.

### Question 4
Code the descent gradient algorithm with J = (h - yi), the error

In [ ]:
alpha = 0.01      # Learning parameter
Nsamples = 100000  # Number of samples

# Training set
x = np.random.rand(3, Nsamples)
x[0] = np.ones(Nsamples)
y = f_deterministic(x)

# Random initialisation of matrices
Theta0 = np.random.normal(scale=1.0,size=(Ns[1],Ns[0]))
Theta1 = np.random.normal(scale=1.0,size=(Ns[2],Ns[1]))
Theta2 = np.random.normal(scale=1.0,size=(Ns[3],Ns[2]))
Thetas = (Theta0, Theta1, Theta2)

J = np.zeros(Nsamples)
error = 1
for t in range(Nsamples):
    # TODO : put your code here, use with J = (h - yi) for the error


    J[t] = np.random.randn()


# FIGURES : plot the real function, the network approximation and the error evolution   


### Mini-batch gradient descent

Now, we want to implement a mini-batch gradient descent, where at each step, $N_{batch}$ samples are drawn at random from the training set. 

### Question 5
Code the mini-batch gradient descent

In [ ]:
alpha = 0.05       # Learning parameter
Nsamples = 100000  # Number of samples
Nbatch = 16        # Size of mini-batch
Nstep = 32000      # Number of mini-batches for training

# Random initialisation of matrices
Theta0 = np.random.normal(scale=1.0,size=(Ns[1],Ns[0]))
Theta1 = np.random.normal(scale=1.0,size=(Ns[2],Ns[1]))
Theta2 = np.random.normal(scale=1.0,size=(Ns[3],Ns[2]))
Thetas = (Theta0, Theta1, Theta2)

In [ ]:
# Random initialisation of matrices
Theta0 = np.random.normal(scale=1.0,size=(Ns[1],Ns[0]))
Theta1 = np.random.normal(scale=1.0,size=(Ns[2],Ns[1]))
Theta2 = np.random.normal(scale=1.0,size=(Ns[3],Ns[2]))
Thetas = (Theta0, Theta1, Theta2)

J = np.zeros(Nstep)
error = 1
for t in range(Nstep):
# Your code here
